In [1]:
from manim import *
import networkx as nx
import numpy as np
import copy

Manim Community v0.16.0.post0

In [2]:
def crea_matriz_imagenes_arbol():
    matriz = [[SVGMobject("imagenes/comida/plato.svg").scale(1.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
               SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
               SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champiñon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1), 
                SVGMobject("imagenes/comida/platillos/fusili-pom.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham.svg").scale(.6)],
               [SVGMobject("imagenes/comida/macaroni.svg").rotate(-.2), 
                SVGMobject("imagenes/comida/platillos/macaroni-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/macaroni-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-cham.svg").scale(.7)]]

    return matriz



In [3]:
def escalar_submatriz(submatriz):
    for i in range(len(submatriz)):
        for j in range(len(submatriz[i])):
            submatriz[i][j].scale(0.5)

def construir_mobject_tabla(comb, primera_columna, primer_renglon):
    return MobjectTable(
        comb,
        row_labels= primera_columna,
        col_labels= primer_renglon,
        include_outer_lines=True
    )

def construir_tabla(self, matriz, tipos_1, tipos_2):
    submatriz = [[matriz[i][j] for j in tipos_2] for i in tipos_1]
    escalar_submatriz(submatriz)
    primera_columna = [submatriz[i][0] for i in range(1, len(submatriz))]
    primer_renglon = submatriz[0][1:]
    comb = [row[1:] for row in submatriz[1:]]
    tabla = construir_mobject_tabla(comb, primera_columna, primer_renglon)
    return tabla


In [4]:
def distribuir_puntos_uniformemente(n, dist):
    puntos = []
    if n % 2 == 0:  # Si n es par, se divide niformemente a ambos lados de 0.
        cant_puntos_lado = n // 2
        puntos_lado = [dist * (i - 1/2) for i in range(1, cant_puntos_lado + 1)]
    else:  # Si n es impar, se coloca un punto en 0 y se divide el resto uniformemente.
        cant_puntos_lado = (n - 1) // 2
        puntos_lado = [dist * i for i in range(1, cant_puntos_lado + 1)]
        puntos += [0]
    
    # Distribuye puntos uniformemente arriba y abajo de 0 con una distancia de 0.5.
    
    puntos.extend(puntos_lado)
    puntos.extend(-1* np.array(puntos_lado))

    return sorted(puntos, reverse=True)

In [5]:
def generar_puntos_y_aristas_pisos_vertical(puntos_antes, p, arreglo, x, distancia_pisos):
    puntos = []
    puntos.append(p)
    aristas =[]
    for dec in reversed(arreglo):
        puntos_r = []
        coor_ren = []
        x -= distancia_pisos
        for i in range(0, len(puntos_antes), dec):
            valor_y = round((puntos_antes[i] + puntos_antes[i + dec - 1]) / 2,2)
            puntos_r += [valor_y]
            coor_ren.append((x, valor_y, 0))
            for j in range(i , i+ dec):
                par = [(x, -1 * valor_y, -5), (x + distancia_pisos, -1 * puntos_antes[j],-5)]
                aristas.insert(0, par)

        puntos.insert(0, coor_ren)
        puntos_antes = puntos_r
        
    return puntos, aristas

def distribucion_vertical(arreglo, comb_totales, dist = .8):
    pisos = len(arreglo)
    distancia_pisos = 2
    x = pisos * distancia_pisos -6 # eje es x
    puntos_ultimo_renglon = distribuir_puntos_uniformemente(comb_totales, dist)
    puntos_antes = puntos_ultimo_renglon
    p = [(x, valor, 0) for valor in puntos_antes]
    
    puntos, aristas = generar_puntos_y_aristas_pisos_vertical(puntos_antes, p, arreglo, x, distancia_pisos)
    
    return (puntos, aristas)


In [6]:
def generar_puntos_y_aristas_pisos_horizontal(puntos_antes, p, arreglo, y, distancia_pisos):
    puntos = []
    puntos.append(p)
    aristas = []
    for dec in reversed(arreglo):
        puntos_r = []
        coor_ren = []
        y += distancia_pisos
        for i in range(0, len(puntos_antes), dec):
            valor_x = round((puntos_antes[i] + puntos_antes[i + dec - 1])/2,2)
            puntos_r += [valor_x]
            coor_ren.append((valor_x,y, 0))
            for j in range(i , i + dec):
                par = [(-1 * valor_x, y, -5), (-1 * puntos_antes[j], y - distancia_pisos, -5)]
                aristas.insert(0, par)

        puntos.insert(0, coor_ren)
        puntos_antes = puntos_r
    
    return puntos, aristas
    

def distribucion_horizontal(arreglo, comb_totales, dist = None):
    if dist is None:
        dist = 12 / comb_totales
    pisos = len(arreglo)
    distancia_pisos = 1.5 if pisos <= 4 else 1
    y = 3 - pisos * distancia_pisos # eje es y
    puntos_ultimo_renglon = distribuir_puntos_uniformemente(comb_totales, dist)
    puntos_antes = puntos_ultimo_renglon
    p = [(valor, y, 0) for valor in puntos_antes]
    
    puntos, aristas = generar_puntos_y_aristas_pisos_horizontal(puntos_antes, p, arreglo, y, distancia_pisos)
    
    return (puntos, aristas)


In [7]:
def multiplicar_valores(arreglo, dist = .8):
    if len(arreglo) == 0:
        return 0  # Si el arreglo está vacío, el resultado es 0.
    else:
        comb_totales = np.prod(arreglo)
        if comb_totales <= 10: # vertical
            # dist = .8
            puntos,aristas = distribucion_vertical(arreglo, comb_totales, dist)

        else:
            puntos, aristas = distribucion_horizontal(arreglo, comb_totales, dist)
        
        return puntos, aristas

In [8]:
def transpose_matrix(matrix):
    transpuesta = [[0] * len(matrix) for _ in range(len(matrix[0]))]

    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            transpuesta[j][i] = matrix[i][j]

    return transpuesta

In [9]:
def comb_pasta_sencilla(matriz_completa,tipos_1, tipos_2):
    submatriz = [[matriz_completa[i][j] for j in tipos_2] for i in tipos_1]
    tipos_2 = submatriz[0][1:]
    combinaciones = [fila[0] for fila in submatriz]
    
    for i in range(1, len(tipos_1)):
        copias_tipos_2 =[copy.deepcopy(combinacion) for combinacion in tipos_2]
        combinaciones.extend(copias_tipos_2)
    
    for j in range(1, len(tipos_1)):
        combinaciones.extend(submatriz[j][1:])
            
    combinaciones
    
    return combinaciones

In [ ]:
def construir_arbol(matriz_completa,tipos_1,tipos_2, dist = .8):
    valores = crear_valores(tipos_1, tipos_2)
    coordenadas, aristas = multiplicar_valores(valores, dist)
    
    combinaciones = comb_pasta_sencilla(matriz_completa, tipos_1, tipos_2)
    
    arbol_niveles, arbol, aristas = construir_pisos_arbol(coordenadas, combinaciones, aristas, valores)

    return (arbol_niveles, arbol, coordenadas, aristas)

def crear_valores(tipos_1, tipos_2):
    dec_1 = len(tipos_1) -1
    dec_2  = len(tipos_2) -1

    valores = [dec_1, dec_2, 1]
    return valores

def construir_pisos_arbol(coordenadas, combinaciones, aristas, valores):
    arbol_niveles = []
    arbol = VGroup()
    i = 0
    j=0
    k = 0
    num_aristas = 1
    for piso in coordenadas:
        nivel = VGroup()
        for punto in piso:
            escala = 0.3 if k != 1 else 0.4
            combinaciones[i].move_to(punto).scale(escala)
            arbol.add(combinaciones[i])
            nivel.add(combinaciones[i])
            i +=1 
        if k < len(valores):
            aristas, num_aristas, k, j = agregar_aristas(valores, aristas, num_aristas, arbol, nivel, k, j)

        arbol_niveles.append(nivel)
        k += 1
        
    return arbol_niveles, arbol, aristas

def posicionar_imagenes(combinaciones, arbol, nivel, k, punto):
    escala = 0.3 if k != 1 else 0.4
    combinaciones[i].move_to(punto).scale(escala)

def agregar_aristas(valores, aristas, num_aristas, arbol, nivel, k, j):
    num_aristas *= valores[k]
    r = 0
    while j < len(aristas) and r < num_aristas:
        linea = Line(start = aristas[j][0], end = aristas[j][1])
        linea.z_index = -5
        arbol.add(linea)
        nivel.add(linea)
        j += 1
        r += 1
    return aristas, num_aristas, k, j

In [11]:
def crear_arbol_niveles(self,arbol_niveles):
    for obj in arbol_niveles:
        group = VGroup(*obj)
        self.add(group)
        self.play(FadeIn(group))

In [30]:
def imagenes_arbol_complicado():
    matriz = [[[SVGMobject("imagenes/comida/plato.svg").scale(1.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
               SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
               SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champiñon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1), 
                SVGMobject("imagenes/comida/platillos/fusili-pom.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham.svg").scale(.6)]],
             [[SVGMobject("imagenes/comida/salmon.svg").scale(.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
              SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
              SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champiñon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo-salmon.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham-salmon.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1),
                SVGMobject("imagenes/comida/platillos/fusili-pom-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo-salmon.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham-salmon.svg").scale(.6)]],
             [[SVGMobject("imagenes/comida/carne.svg").scale(.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
              SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
              SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champiñon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo-carne.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham-carne.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1),
                SVGMobject("imagenes/comida/platillos/fusili-pom-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo-carne.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham-carne.svg").scale(.6)]]]
    return matriz


In [ ]:
def comb_pasta(matriz_completa, tipos_1, tipos_2, tipos_3):
    submatriz = [[[matriz_completa[i][j][k] for k in tipos_2] for j in tipos_1] for i in tipos_3]
    n = len(tipos_3) -1
    p = len(tipos_2) -1
    k = len(tipos_1) -1
    valores = [k, p, n,1]

    tipos_1, tipos_2, tipos_3 = ajustar_tipos(submatriz)

    combinaciones = [matriz_completa[0][0][0]]
    combinaciones.extend(tipos_1)

    combinaciones = crear_matriz_copias(tipos_1, tipos_2, tipos_3, submatriz, combinaciones, k , p ,n)
    
    return combinaciones, valores

def ajustar_tipos(submatriz):
    tipos_1 = [submatriz[0][i][0] for i in range(1, len(submatriz[0]))]
    
    tipos_2 = submatriz[0][0][1:]

    tipos_3 = [submatriz[i][0][0] for i in range(1, len(submatriz))]
    
    return tipos_1, tipos_2, tipos_3
    
def crear_matriz_copias(tipos_1, tipos_2, tipos_3, submatriz, combinaciones, k , p ,n):

    for i in range(0, len(tipos_1)):
        copias_tipos2 =copy.deepcopy(tipos_2)
        combinaciones.extend(copias_tipos2)

    for i in range(0, len(tipos_1) * len(tipos_2)):
        copias_tipos3 = copy.deepcopy(tipos_3)
        combinaciones.extend(copias_tipos3)

    for i in range(0, k):
        for r in range(0, p):
            for j in range(0,n):
                copia_tipos3 = copy.deepcopy(submatriz[j+1][(i+1)][r+1])
                combinaciones.append(copia_tipos3)
    return combinaciones

In [8]:
def construir_arbol_compl(matriz_completa, tipos_1, tipos_2, tipos_3, dist = .8):
    dec1 = len(tipos_1) - 1
    dec2 = len(tipos_2) - 1
    dec3 = len(tipos_3) - 1
    dec = [dec1,dec2,dec3,1]
    combinaciones, valores = comb_pasta(matriz_completa, tipos_1, tipos_2, tipos_3)
    coordenadas, aristas = multiplicar_valores(valores, dist)
    
    arbol_niveles, arbol, aristas = construir_pisos_arbol_compl(coordenadas, combinaciones, aristas, valores)
    

    return (arbol_niveles, arbol, coordenadas, aristas)

def construir_pisos_arbol_compl(coordenadas, combinaciones, aristas, dec):
    arbol_niveles = []
    arbol = VGroup()
    i = 0
    j=0
    k = 0
    num_aristas = 1
    for piso in coordenadas:
        nivel = VGroup()
        for punto in piso:
            escala = 0.25 if k != 1 else 0.3
            combinaciones[i].move_to(punto).scale(escala)
            arbol.add(combinaciones[i])
            nivel.add(combinaciones[i])
            i +=1 
        if k < len(dec):
            num_aristas *= dec[k]
            r = 0
            while j < len(aristas) and r < num_aristas:
                linea = Line(start = aristas[j][0], end = aristas[j][1])
                linea.z_index = -5
                #self.play(Write(edge))
                arbol.add(linea)
                nivel.add(linea)
                j += 1
                r += 1
            
        arbol_niveles.append(nivel)
        k += 1
    return arbol_niveles, arbol, aristas

In [ ]:
def animar_arbol_sencillo(self, fusilli, salsa_pom, salsa_par, salsa_ajo):
    """
    Reacomoda los objetos con las imagenes de las salsas y de la pasta fusili y crea lineas entre ellos para convertirlo 
    en un arbol de decision. Luego crea imagenes con el platillo resultante de cada camino y los une con lineas al arbol.

    Args:
        fusili (SVGMobject): Objeto con la imagen de la pasta fusili 
        salsa_pom, salsa_par, salsa_ajo (SVGMobject): Objetos con imagenes de las pastas
    
    Return:
        grupo_arbol (VGroup): vector que agrupa todos los objetos visuales del arbol rendereado

    """

    self.play(fusilli.animate.move_to([-5, 0, 0]), 
            salsa_pom.animate.move_to([1, -2.5, 0]).scale(.7),
            salsa_par.animate.move_to([1, 0, 0]).scale(.7),
            salsa_ajo.animate.move_to([1, 2.5, 0]).scale(.7), 
            run_time = 1
            )
    
    lineas_a_salsas = _crear_lineas_de_fusilli_a_salsas(self, fusilli, salsa_pom, salsa_par, salsa_ajo)
    
    # Crear imágenes con los platillos resultantes de cada camino del arbol
    platillos = _crear_platillos(self, )

    lineas_a_platillos = _crear_lineas_a_platillos(self, salsa_pom, salsa_par, salsa_ajo, *platillos)
    
    # Agrupar todos los elementos del árbol
    grupo_arbol = VGroup(fusilli, salsa_pom, salsa_par, salsa_ajo, *platillos,
            *lineas_a_salsas, *lineas_a_platillos)
    
    return grupo_arbol

def _crear_lineas_de_fusilli_a_salsas(self, fusilli, salsa_pomodoro, salsa_parmesano, salsa_ajo):
    lineas = [
        Line(start=fusilli.get_center(), end=salsa.get_center()).set_z_index(-1)
        for salsa in (salsa_pomodoro, salsa_parmesano, salsa_ajo)
    ]
    self.play(*(Create(linea) for linea in lineas))
    return lineas

def _crear_platillos(self, ):
    fus_pom = SVGMobject("imagenes/comida/platillos/fusili-pom.svg").move_to([4.5, -2.5, 0]).scale(.5)
    fus_par = SVGMobject("imagenes/comida/platillos/fusili-par.svg").move_to([4.5, 0, 0]).scale(.5)
    fus_ajo = SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").move_to([4.5, 2.5, 0]).scale(.5)
    self.play(Write(fus_pom), Write(fus_par),Write(fus_ajo),  run_time=.9)
    
    return fus_pom, fus_par, fus_ajo

def _crear_lineas_a_platillos(self, salsa_pomodoro, salsa_parmesano, salsa_ajo, plato_pomodoro, plato_parmesano, plato_ajo):
    salsas = [salsa_pomodoro, salsa_parmesano, salsa_ajo]
    platillos = [plato_pomodoro, plato_parmesano, plato_ajo]
    lineas = [
        Line(start=salsa.get_center(), end=platillo.get_center()).set_z_index(-1)
        for salsa, platillo in zip(salsas, platillos)
    ]
    self.play(*(Create(linea) for linea in lineas), tiempo_ejecucion=.5)
    return lineas